# Downloading US trade data from the US Census API in R

This notebook shows how to use the `censusapi` package in R to access US data on imports and exports

## Install and load libraries
This checks if the required libraries are installed, if not they are downloaded. If so, they are loaded.

In [22]:
packages <- c("censusapi", "dplyr", "tidyr", "stringr")
new_packages <- packages[!(packages %in% installed.packages()[,"Package"])]
if(length(new_packages)) install.packages(new_packages)

# the semicolon on the end of this line supresses warning messages
invisible(lapply(packages, library, character.only = TRUE));

## Load your Census API key and store it

You'll need an API key from the Census Bureau. If you don't have one you can request one here https://api.census.gov/data/key_signup.html

You'll want to [save that key as an environment variable](https://github.com/Quartz/how-we-make-things/blob/master/cookbook-examples/how-to-create-an-environment-variable.md) called `US_CENSUS_API_KEY`

In [3]:
Sys.setenv(CENSUS_KEY=Sys.getenv("US_CENSUS_API_KEY"))

## Downloading the data

### Specify the exact data you're after
Consult the Census's documentation on which data_columns you want in the API docs https://www.census.gov/foreign-trade/reference/guides/Guide%20to%20International%20Trade%20Datasets.pdf (pgs 19-25)

In [33]:
# Let's get monthly data for 2018
years <- c(2018)
months <- 1:12

# save an empty list to hold our year_month strings
year_months <- c()

# loop through every year in the years list
for (year in years) {
    # loop through every month in the months list
    for (month in months) {
        # create a year_month string
        year_month <- paste(year, str_pad(month,2, pad="0"), sep="-")
        
        # add the current year_month to the year_months array list
        year_months <- c(year_months, year_month)
    }
}

# print out the time periods were using
print(year_months)

# Important!: Some data columns require other data columns. 
# The requirements are listed in the API docs. eg. "I_COMMODITY_LDESC"
# requires "I_COMMODITY"
data_columns = c(
    "I_COMMODITY",       # this is the HS code
    "I_COMMODITY_LDESC", # the long description of the commodity
    "GEN_VAL_MO"         # General Imports, Total Value
)

 [1] "2018-01" "2018-02" "2018-03" "2018-04" "2018-05" "2018-06" "2018-07"
 [8] "2018-08" "2018-09" "2018-10" "2018-11" "2018-12"


### Download 6-digit imports from anywhere

In [32]:
all_results <- c()
for (t in year_months) {
    
    print(paste("Getting", t, "from the API"))
    flush.console() # this print immediately, not on loop completion
    
    results <- getCensus(name = "timeseries/intltrade/imports/hs",
                     vars=data_columns,
                     t=t,
                     COMM_LVL="HS6",
                     SUMMARY_LVL2="HS" # aggregate on HS code
                     )
    
    if (length(all_results) == 0) {
        # if the frame doesn't exist, create it
        all_results <- results
    }
    else {
        # if the frame exists, append to it
        all_results <- bind_rows(all_results, results)
        
    }
}

head(all_results)

[1] "Getting 2018-01 from the API"
[1] "Getting 2018-02 from the API"
[1] "Getting 2018-03 from the API"
[1] "Getting 2018-04 from the API"
[1] "Getting 2018-05 from the API"
[1] "Getting 2018-06 from the API"
[1] "Getting 2018-07 from the API"
[1] "Getting 2018-08 from the API"
[1] "Getting 2018-09 from the API"
[1] "Getting 2018-10 from the API"
[1] "Getting 2018-11 from the API"
[1] "Getting 2018-12 from the API"


time,I_COMMODITY,I_COMMODITY_LDESC,GEN_VAL_MO,COMM_LVL,SUMMARY_LVL2
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2018-01,010121,"HORSES, LIVE, PUREBRED BREEDING",8018067,HS6,HS
2018-01,010129,"HORSES, LIVE, OTHER THAN PUREBRED BREEDING",58368217,HS6,HS
2018-01,010221,"CATTLE, LIVE, PUREBRED BREEDING",132409,HS6,HS
2018-01,010229,"CATTLE, LIVE, OTHER THAN PUREBRED BREEDING",97912906,HS6,HS
2018-01,010231,"BUFFALO, LIVE, PUREBRED BREEDING",2399,HS6,HS
2018-01,010239,"BUFFALO, LIVE, OTHER THAN PUREBRED BREEDING",4627131,HS6,HS


### Download 6-digit exports to China and India
Census country codes are listed here: https://www.census.gov/foreign-trade/schedules/c/country.txt

Notice the different `name` in the `getCensus` call and the different `data_column` names

In [55]:
data_columns_for_exports = c(
    "E_COMMODITY",       # this is the HS code
    "E_COMMODITY_LDESC", # the long description of the commodity
    "CTY_CODE",          # country code
    "CTY_NAME",          # country name
    "ALL_VAL_MO"         # Total Value
)

country_codes = c(
    "5700", # China
    "5330"  # India
)

all_results <- c()
for (t in year_months) {
    for (c in country_codes) {
        print(paste("Getting", paste0(c, "’s"), t, "from the API"))
        flush.console() # this print immediately, not on loop completion
    
        results <- getCensus(name = "timeseries/intltrade/exports/hs",
                         vars=data_columns_for_exports,
                         t=t,
                         COMM_LVL="HS6",
                         CTY_CODE=c
                         )

        if (length(all_results) == 0) {
            # if the frame doesn't exist, create it
            all_results <- results
        }
        else {
            # if the frame exists, append to it
            all_results <- bind_rows(all_results, results)

        }
        
    }
}

head(all_results)

[1] "Getting 5700’s 2018-01 from the API"
[1] "Getting 5330’s 2018-01 from the API"
[1] "Getting 5700’s 2018-02 from the API"
[1] "Getting 5330’s 2018-02 from the API"
[1] "Getting 5700’s 2018-03 from the API"
[1] "Getting 5330’s 2018-03 from the API"
[1] "Getting 5700’s 2018-04 from the API"
[1] "Getting 5330’s 2018-04 from the API"
[1] "Getting 5700’s 2018-05 from the API"
[1] "Getting 5330’s 2018-05 from the API"
[1] "Getting 5700’s 2018-06 from the API"
[1] "Getting 5330’s 2018-06 from the API"
[1] "Getting 5700’s 2018-07 from the API"
[1] "Getting 5330’s 2018-07 from the API"
[1] "Getting 5700’s 2018-08 from the API"
[1] "Getting 5330’s 2018-08 from the API"
[1] "Getting 5700’s 2018-09 from the API"
[1] "Getting 5330’s 2018-09 from the API"
[1] "Getting 5700’s 2018-10 from the API"
[1] "Getting 5330’s 2018-10 from the API"
[1] "Getting 5700’s 2018-11 from the API"
[1] "Getting 5330’s 2018-11 from the API"
[1] "Getting 5700’s 2018-12 from the API"
[1] "Getting 5330’s 2018-12 from t

time,E_COMMODITY,E_COMMODITY_LDESC,CTY_CODE,CTY_NAME,ALL_VAL_MO,COMM_LVL,CTY_CODE_1
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
2018-01,010221,"CATTLE, LIVE, PUREBRED BREEDING",5700,CHINA,9544,HS6,5700
2018-01,010410,"SHEEP, LIVE",5700,CHINA,19230,HS6,5700
2018-01,010599,"TURKEYS, DUCKS, GEESE AND GUINEA FOWLS, LIVE, WEIGHING MORE THAN 185 G (6.53 OZ.) EACH",5700,CHINA,30222,HS6,5700
2018-01,010619,"MAMMALS, LIVE, NESOI",5700,CHINA,84064,HS6,5700
2018-01,020120,"MEAT OF BOVINE ANIMALS, CUTS WITH BONE IN (OTHER THAN HALF OR WHOLE CARCASSES), FRESH OR CHILLED",5700,CHINA,140321,HS6,5700
2018-01,020130,"MEAT OF BOVINE ANIMALS, BONELESS, FRESH OR CHILLED",5700,CHINA,555947,HS6,5700


### Download imports of Women's elastic knit trousers and Women's blue jeans since 2013

In [52]:
data_colums_by_quantity = c(
    "I_COMMODITY",       # this is the HS code
    "GEN_QY1_MO"         # General Imports, Total Value
)

results <- getCensus(name = "timeseries/intltrade/imports/hs",
                     vars=data_colums_by_quantity,
                     I_COMMODITY="6204624010", # jeans
                     I_COMMODITY="6204621511", # jeans
                     I_COMMODITY="6204624011", # jeans
                     I_COMMODITY="6204628011", # jeans
                     I_COMMODITY="6104632006", # yoga pants
                     time="from 2013-01",      # this is another way to get multiple months
                     SUMMARY_LVL2="HS")

head(results)

time,I_COMMODITY,GEN_QY1_MO,I_COMMODITY_1,SUMMARY_LVL2
<chr>,<chr>,<dbl>,<dbl>,<chr>
2013-01,6104632006,536464,6104632006,HS
2013-01,6204624011,1352649,6204624011,HS
2013-02,6104632006,464316,6104632006,HS
2013-02,6204624011,1121734,6204624011,HS
2013-03,6104632006,362278,6104632006,HS
2013-03,6204624011,791531,6204624011,HS


#### Reformat the data
As you can see, this data is in a long format, it also has multiple identifiers for blue jeans because the code has changed over time. Here's how to take care of that

In [53]:
results %>% 
    # select certain columns
    select("time", "I_COMMODITY", "GEN_QY1_MO") %>%
    
    # pivot the table
    spread("I_COMMODITY", "GEN_QY1_MO") %>%          
    
    # add `HS` to the column names (skipping the first)
    rename_at(vars(-(0:1)), ~ paste0("HS", .)) %>%  

    # replace NAs with 0
    mutate_all(~replace(., is.na(.), 0)) %>%        
    
    # sum the three denim codes into a single column
    rowwise() %>%
        mutate(                                     
            all_denim_doz = sum(HS6204621511, HS6204624011, HS6204628011, na.rm = TRUE)
        ) %>%

    # rename the elastics knits HS code
    rename(elastic_knits_doz = HS6104632006) %>%  

    # select only the renamed and time columns
    select("time", "elastic_knits_doz", "all_denim_doz") %>%

    # add the first of the month to the date
    mutate(time = paste0(time,"-01"))               

time,elastic_knits_doz,all_denim_doz
<chr>,<dbl>,<dbl>
2013-01-01,536464,1352649
2013-02-01,464316,1121734
2013-03-01,362278,791531
2013-04-01,405148,904684
2013-05-01,434905,1221976
2013-06-01,529142,1663770
2013-07-01,761196,2488386
2013-08-01,780402,1836866
2013-09-01,779895,1406771
